In [1]:
import os
import time
import requests
from selenium import webdriver

In [2]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)


    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        
        results_start = len(thumbnail_results)

    return image_urls

In [3]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [4]:
def search_and_download(search_term: str, driver_path: str, target_path='./pizzaimg', number_images=30):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' '))) 
    if not os.path.exists(target_folder):
        os.makedirs(target_folder) 

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

In [ ]:
DRIVER_PATH = './chromedriver'
search_term = 'pizza'
search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100
Found: 31 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRRwEZByYkG3EK7ktr4vSoZX31KdjQyJ0YIdA&usqp=CAU - as ./pizzaimg\pizza
SUCCESS - saved https://static.toiimg.com/photo/53110049.cms - as ./pizzaimg\pizza
SUCCESS - saved https://i.ytimg.com/vi/56eRAEY1t8A/maxresdefault.jpg - as ./pizzaimg\pizza
SUCCESS - saved https://storage.googleapis.com/gen-atmedia/3/2012/10/cb2e9502cd9da3468caa944e15527b19bce68a8e.jpeg - as ./pizzaimg\pizza
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS1TCu8aHouyGcTZVvLmHs825qXdAFecEdbig&usqp=CAU - as ./pizzaimg\pizza
SUCCESS - saved https://www.qsrmagazine.com/sites/default/files/styles/story_page/public/story/pizza-hut-turns-comeback-expert_0.jpg?itok=U_V-5YAD - as ./pizzaimg\pizza
